<html>
  <head>
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.1.0/css/bootstrap.min.css">
    <style>
        pre {
            align-items: center;
            justify-content: center;
            display: flex;
            background: white;
        }   
    </style>
</head>
</html>

In [1]:
from wordGraph.wordGraph import WordGraph
from wordGraph.io.pubmed_io import parse_pubmed_api
from wordGraph.visualization.viz_utils import plot_top_terms, plot_graph

from IPython.display import display, HTML, clear_output, Image
from ipywidgets import HTML, VBox, HBox, Label, widgets, Layout

In [2]:
## Header logos
image1 = Image('assets/logo_univ.png')
image2 = Image('assets/sogeti.png')
image3 = Image('assets/centre_borelli.png')

img1 = widgets.Image(
    value=image1.data,
    width='auto',
    height='auto'
)

img2 = widgets.Image(
    value=image2.data,
    width='auto',
    height='auto'
)

img3 = widgets.Image(
    value=image3.data,
    width='auto',
    height='auto',
)

HBox(
    [img1,img3,img2], 
    layout={'display':'flex','height':'100px','grid_gap':'300px', 'justify_content':'center'}
)

In [3]:
## Default parameters
MAX_DOC = 10000
N_CLUST = 3
MAX_NEIGH = 50
MAX_TOP_TERMS = 50
N_SHUFFLES=100
CONF_THRESHOLD=.01

In [4]:
%%html
<!--Define personalized box styles-->
<style>
.box_style{
    box-shadow: 0px 0px 0px 0 rgb(0 0 0 / 0%), 0 3px 7px 0 rgb(0 0 0 / 24%);
    border-radius: 10px
}
.mini_box_style{
    box-shadow: 0px 0px 0px 0 rgb(0 0 0 / 0%), 0 1px 6px 0 rgb(0 0 0 / 24%);
    border-radius: 10px
}
.button_style{
    border-radius: 5px;
}
</style>

In [5]:
## I. Initialize panels 

# A. Initialize main panel
out = widgets.Output(layout={'display':'flex',
                             'flex_flow':'column',
                             'align_items':'center',
                             'justify_content':'center'
                            })
# B. Initialize a panel to plot graphs
graph_out = widgets.Output(layout={
                             'flex_flow':'column',
                             'justify_content':'center'
                            })

In [6]:
## II. Fill main panel with parameters

# pubmed and coclust box layout
box_layout = {'margin':'10px 10px 10px 10px', 'padding':'18px'}

# A. Pubmed query
query_label = Label(value='PubMed Query :')
query = widgets.Text(value='covid', disabled=False).add_class('button_style')
pm_query = VBox([query_label,query])

# B. Number of documents
label = Label(value='Number of documents :')
n_doc = widgets.BoundedFloatText(    
    min=100,
    max=MAX_DOC,
    step=100,
    disabled=False)
n_doc.value=1500
num_doc = VBox([label,n_doc])

# C. Confirm button
cc_confirm = widgets.Button(
            description='Start co-clustering',
            disabled=False,
            button_style='info',
            layout={'margin':'0 0 10px 0'}
        )

# D. Number of clusters
nc_label = Label(value='Number of clusters :')
nc = widgets.IntSlider(min=2, max=20, value=N_CLUST)
nc.style.handle_color = 'lightblue'
num_clust = VBox([nc_label,nc])

# E. Number of top terms
ntt_label = Label(value='Number of top terms :')
ntt = widgets.IntSlider(min=5, max=MAX_TOP_TERMS, value=20)
ntt.style.handle_color = 'lightblue'
num_top_terms = VBox([ntt_label,ntt])

# F. Group pubmed & coclust boxes
params_box = HBox([
    # pubmed box
    VBox([ 
        HTML('<h5>Pubmed<h5>'),
        VBox([ 
            pm_query, 
            VBox([num_doc]),
        ], layout=box_layout).add_class("mini_box_style")],
        layout={'align_items':'center'}),
    # coclust box
    VBox([ 
        HTML('<h5>Coclustering<h5>'),
        VBox([ 
            num_clust, num_top_terms
        ], layout=box_layout).add_class("mini_box_style")],
        layout={'align_items':'center'})
],layout={ 'justify_content':'center', 'grid_gap':'80px', 'margin' : '0px 0 30px 0',})

# G. Final parameters box
params = VBox([ 
    HTML('<h3>Parameters</h3>'),
    params_box,
    cc_confirm
], layout={'align_items':'center',
           'padding':'20px 50px 20px 50px',
           'margin' : '20px 20px 30px 20px', 
           'background_color':'white',
           'width': '-webkit-fill-available'
          })

In [7]:
## III. Graph parameters

# A. Coclust choice box
# A.1. Cocluster choice layout
layout_gp = Layout(
    display='flex',
    flex_flow='column',
    align_items='center',
    justify_content='center',
    margin='10px 0 30px 20px',
    grid_gap='20px',
)

# A.2. Cocluster choice buttons
choice = widgets.ToggleButtons(
    options=['Cluster '+str(i+1) for i in range(nc.value)],
    description='',
    disabled=False,
    button_style='', 
    style={"button_width": "100px"},
)
cc_choice = VBox([
    HTML('<h5>Cluster choice :</h5>'),
    choice
], layout={'align_items':'center','grid_gap':'10px',}
)

# A.3. Confirm button
confirm = widgets.Button(
            description='Generate graph',
            disabled=False,
            button_style='success',
        )

# A.4. Clear button
clear = widgets.Button(
            description='🧹',
            disabled=False,
            button_style='',
            layout={'width':'50px'}
        ).add_class('button_style')

## B. MIIC parameters
#  B.1. conf_threshold & n_shuffle layout
layout_miic = Layout(
    grid_gap='20px',
    margin='10px 10px 10px 10px',
    justify_content='center',
    padding='20px 20px 30px 20px'
)

# B.2. Number of shuffles
ns_label = Label(value='Num. shuffles :')
ns = widgets.BoundedIntText(
    value=N_SHUFFLES,
    step=100,
    min=100,
    max=5000,
    layout = Layout(width='80%')
)
n_shuffle = VBox([ns_label,ns],layout={'align_items':'center'})

# B.3. Confidence threshold
ct_label = Label(value='Confidence threshold :')
ct = widgets.BoundedFloatText(
    value=CONF_THRESHOLD,
    min=0.01,
    max=1,
    step=0.01,
    layout = Layout(width='70%')
)
conf_threshold = VBox([ct_label,ct],layout={'align_items':'center'})

# B.4 Group MIIC parameters (n-shuffles & conf-threshold) 
graph_params = VBox([
    HTML('<h5>Graph parameters</h5>'),
    HBox([conf_threshold, n_shuffle], layout = layout_miic).add_class('box_style')],
    layout={'align_items':'center','width':'40%'}
)

# C. Final graph parameters
coclust_choice = HBox([cc_choice, graph_params, HBox([confirm, clear])],layout = layout_gp)

In [8]:
## IV. Define buttons on_click event functions 

# A. Define funtions
def on_coclust_button_clicked(button):
    button.disabled=True
    global out,spinner
    with out:
        global my_corpus, wg
        parsed, clean_corpus = parse_pubmed_api(query.value,int(n_doc.value))  
        
        ## Call WordGraph
        wg = WordGraph()
        
        ## Perform coclustering
        wg.fit_coclust(clean_corpus, nc.value, verbose=False)
        
        # Add output for top terms plot
        out2 = widgets.Output(layout={'display':'flex',
                             'align_items':'center',
                             'justify_content':'center'
                            })
        display(out2)
        with out2:
            plot_top_terms(wg, nb_terms=10)
        cc_choice.options=['Cluster '+str(i+1) for i in range(nc.value)]
        display(coclust_choice)
        display(graph_out)
        
def on_graph_button_clicked(button):
    button.disabled=True
    global out, wg, graph_out
    try:
            # Set cocluster choice
            wg.set_cocluster(int(choice.value[8:]), nb_terms=ntt.value, verbose=False)
            # Export the new file
            wg.build_graph(n_shuffles=ns.value, conf_threshold=ct.value)
            # plot graph
            with graph_out:
                plot_graph(wg)
    except:
        with graph_out:
            display(VBox([HTML('<h5>No enough documents to reconstruct the graph, please choose another cocluster.</h5>')],layout={'align_items':'center'}))

    button.disabled=False
    
def on_clear_button_clicked(button):
    global graph_out
    with graph_out:
        clear_output()
        
# B. Link buttons to corresponding events
confirm.on_click(on_graph_button_clicked)
cc_confirm.on_click(on_coclust_button_clicked)
clear.on_click(on_clear_button_clicked)

In [9]:
## V. Add css styles to widgets and display the main panel

params.add_class("box_style")
confirm.add_class("button_style")
cc_confirm.add_class("button_style")

vbox = VBox([HTML('<h2>Welcome to WordGraph</h2>')],layout={'align_items':'center'})

with out:
    display(vbox)
    display(params)
    
display(out)

Output(layout=Layout(align_items='center', display='flex', flex_flow='column', justify_content='center'))